### RISK Calculator 
This is project aims to create a calculator that will display the odds for winning a specific battle in the game of RISK. 

#### What is RISK 
RISK is as board games where you take turns and and try to capture territory. The goal of the game is to take over the world. It is called a "game of world domination". 

#### How do you capture territory 
The way to capture territory is both the attacker and defender rolling dice. If the attacker has a higher dice roll the defender the defender loses an army. If the defender ties the or beats the attackers highest die the attackers lose an army. The attacker can attack with up to three die and the defender can defender can defend with up to two die. If the attacker has two dice higher than both the defenders two dice then the defender loses two armies. If the defender has two dice higher than the attacker, the attacker loses two armies. If one of the attackers die is higher than one of the defenders die and one of the defenders die is larger than the second highest attackers die than both attacker and defender lose a die. 
Furthermore, in the game you cannot lose more armies than you roll with, so if an attacker attackes with three armies and the defender defends with one, the defender can lose at most one army. However, this also means that the attacker can lose at most one army. 

#### How to design a calculator
The calculator is designed below, first I generated the probablities for each different combination of the dice roll. That gave me the expected value of the dice rolls. I then wrote a simple function that sums the expected values of each dice roll. I then wrote a function that simulates the battle mechanics of risk and ran a bunch of simulations. 

#### Why RISK 
I like RISK, its my favorite board game. And because its seems like a problem that seems fairly straight forward to figure out. 

In [1]:
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
def calculate_battle_winner(num_attacker_dice, num_defender_dice): 
    # This function figures calcutates how any dice roll will win 
    n = min(num_attacker_dice, num_defender_dice)
    num_attack_array = [] 
    num_defend_array = [] 
    battle_winner = 0 
    for i in range(num_attacker_dice): 
        num_attack_array.append(random.randint(1,6))
    for i in range(num_defender_dice): 
        num_defend_array.append(random.randint(1,6)) 
    num_attack_array.sort()
    num_attack_array.reverse()
    num_defend_array.sort()
    num_defend_array.reverse()
    for i in range(n):
        if num_attack_array[i] <= num_defend_array[i]: 
            battle_winner -= 1
        else: 
            battle_winner += 1 
    return battle_winner

def replay_attack(n, num_att,
 num_def):
    att_win = 0 
    def_win = 0 
    tie = 0 
    for i in range(n):
        battle_win = calculate_battle_winner(num_att, num_def)
        if battle_win == 0: 
            tie += 1 
        elif battle_win < 0: 
            def_win += 1
        else: 
            att_win += 1 
    return {'Attack':att_win, 'Defender':def_win, 'Tie':tie}


In [3]:
n = 1000000
# 3 attackers 2 defenders 
threeTwo = replay_attack(n, 3, 2)

# 3 attackers 1 defender
threeOne = replay_attack(n, 3, 1)

# 2 attackers 2 defenders 
twoTwo = replay_attack(n, 2, 2) 

# 2 attackers 1 defender 
twoOne = replay_attack(n, 2, 1) 


# 1 attackers 2 defender 
oneTwo = replay_attack(n, 1, 2) 

# 1 attacker 1 defender 
oneOne = replay_attack(n, 1, 1) 

In [4]:
df = pd.DataFrame([threeTwo, threeOne, twoTwo, twoOne, oneTwo, oneOne], columns=threeTwo.keys())
df = df.transpose()
df.columns = ['Three Two', 'Three One', 'Two Two', 'Two One', 'One Two', 'One One']
df
fig = plt.figure()
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)


sns.barplot(x=['Attack', 'Defender', 'Tie'], y=df['Three Two'], ax=ax1)
sns.barplot(x=['Attack', 'Defender', 'Tie'], y=df['Three One'], ax=ax2)
sns.barplot(x=['Attack', 'Defender', 'Tie'], y=df['Two Two'], ax=ax3)
sns.barplot(x=['Attack', 'Defender', 'Tie'], y=df['Two One'], ax=ax4)
sns.barplot(x=['Attack', 'Defender', 'Tie'], y=df['One Two'], ax=ax5)
sns.barplot(x=['Attack', 'Defender', 'Tie'], y=df['One One'], ax=ax6)


In [5]:
df['Three Two'] = df['Three Two']/n
df['Three One'] = df['Three One']/n
df['Two Two'] = df['Two Two']/n
df['Two One'] = df['Two One']/n
df['One Two'] = df['One Two']/n
df['One One'] = df['One One']/n
df

,Three Two,Three One,Two Two,Two One,One Two,One One
Attack,0.371732,0.659499,0.227453,0.578833,0.254919,0.416478
Defender,0.293226,0.340501,0.448846,0.421167,0.745081,0.583522
Tie,0.335042,0.000000,0.323701,0.000000,0.000000,0.000000


In [40]:
prob_dict = df.to_dict()
def roll_expected_value(numAttackers, numDefenders, prob_dict): 
    att_dict = { 3:'Three', 2:'Two', 1:'One'}
    def_dict = {2:'Two', 1:'One'}
    prob = att_dict[numAttackers] + " " + def_dict[numDefenders]
    return prob_dict[prob]
    
def calculate_win_expect_probablility(numAttackers, numDefenders, prob_dict): 

    tie_def = 0 
    while((numAttackers > 0) and (numDefenders > 0)):
        att = math.floor(numAttackers) 
        deff =  math.floor(numDefenders)
        if (numAttackers < 3):
            att = att%3 + 1
        else: 
            att = 3 
        if (numDefenders < 2):
            deff = deff%2 + 1
        else: 
            deff = 2
        prob = roll_expected_value(att, deff, prob_dict)
        numAttackers -= prob['Attack']
        numDefenders -= prob['Defender']
        numAttackers -= prob['Tie']
        numDefenders -= prob['Tie']

    if numAttackers < 0 and numDefenders < 0: 
        if numAttackers < numDefenders:
            numDefenders = 1 
        else: 
            numAttackers = 1 
    if numAttackers < 0: 
        numAttackers = 0
    if numDefenders < 0: 
        numDefenders = 0
    return numAttackers, numDefenders

In [59]:
calculate_win_expect_probablility(100, 1000, prob_dict)

(0, 908.89041099999315)

In [65]:
def choose_assign_numbers(numAttackers, numDefenders):
    att = 0 
    deff = 0 
    if numAttackers >= 3: 
        att = 3 
    elif numAttackers == 2: 
        att = 2 
    elif numAttackers == 1: 
        att = 1 
    if numDefenders >= 2: 
        deff = 2
    elif numDefenders == 1: 
        deff = 1 
    return att, deff
    
def risk_simulator(numAttackers, numDefenders): 
    
    att = 0 
    deff = 0
    
    while((numAttackers > 0) and (numDefenders > 0)):
        att, deff = choose_assign_numbers(numAttackers, numDefenders)
        battle_winner = calculate_battle_winner(att, deff)
        if battle_winner == 0:
            numAttackers -= 1
            numDefenders -= 1
        elif battle_winner == -1: 
            numAttackers -= deff
        elif battle_winner == 1: 
            if att == 3: 
                att = 2 
            numDefenders -= att 
    if numAttackers < 0: 
        numAttackers = 0
    if numDefenders < 0: 
        numDefenders = 0 
    return [numAttackers, numDefenders]

In [87]:
def run_simulations(n, trials): 
    arr = [[[] for i in range(0, n+2)] for j in range(0, n+2)]
    for att in range(1, n+1):
        for deff in range(1, n+1): 
            attack_win = 0 
            def_win = 0 
            for i in range(trials): 
                a = risk_simulator(att, deff)
                attack_win += a[0]
                def_win += a[1]
            attack_win = attack_win/trials
            def_win = def_win/trials
            arr[att][deff] = [attack_win, def_win]
    return arr
            

In [ ]:
run_simulations(100, 10000)

In [80]:
n = 3
arr = [[[] for i in range(1, n+1)] for j in range(1, n+1)]
arr[1][1]

[]